In [24]:
from numpy import genfromtxt
from io import StringIO
from time import time
from datetime import datetime
from sqlalchemy import Column, Integer, Float, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import sqlite3

In [14]:
pwd

'/sfs/qumulo/qhome/adc6fs'

In [15]:
def Load_Data(file_name):
    data = genfromtxt(file_name, delimiter=',', skip_header=1, skip_footer=1)
    return data.tolist()

In [31]:
#prices = Load_Data("prices.csv") 

In [17]:
#funds = Load_Data("funds.csv") 

In [18]:
#daily = Load_Data("daily.csv") 

In [ ]:
Base = declarative_base()

class prices(Base):

    __tablename__ = 'prices'
    __table_args__ = {'sqlite_autoincrement': True}

    id = Column(Integer, primary_key=True, nullable=False) 
    date = Column(Date)
    ticker = Column(Float)
    op = Column(Float)
    hi = Column(Float)
    lo = Column(Float)
    close_split_adjusted = Column(Float)
    close_adjusted = Column(Float)
    vol = Column(Float)

if __name__ == "__main__":
    t = time()

    #Create the database
    engine = create_engine('sqlite:///prices.db')
    Base.metadata.create_all(engine)

    #Create the session
    session = sessionmaker()
    session.configure(bind=engine)
    s = session()

    try:
        file_name = "prices.csv" 
        data = Load_Data(file_name) 

        for i in data:
            record = Price_History(**{
                'date' : datetime.strptime(i[0], '%d-%b-%y').date(),
                "ticker" : i[1],
                'op' : i[2],
                'hi' : i[3],
                'lo' : i[4],
                'close_split_adjusted' : i[5],
                'close_adjusted' : i[6],
                'vol' : i[7]
            })
            s.add(record) #Add all the records

        s.commit() #Attempt to commit all the records
    except:
        s.rollback() #Rollback the changes on error
    finally:
        s.close() #Close the connection
    print ("Time elapsed: " + str(time() - t) + " s.") #0.091s

In [29]:
con = sqlite3.connect('prices.db')
cur = con.cursor()

In [30]:
for row in cur.execute('SELECT * FROM prices'):
        print(row)